# ADM Homework 3

## Search Engine

In [2]:
if True:
    %pip install -qqq numpy scipy matplotlib pandas scikit-learn nltk forex_python babel

Note: you may need to restart the kernel to use updated packages.


In [3]:
import string
import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import re
from forex_python.converter import CurrencyRates
from babel.numbers import get_currency_symbol
from nltk.stem import *
from collections import Counter
from functools import reduce
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import heapq
import numpy as np
from collections import defaultdict

In [10]:
nltk.download('stopwords')
nltk.download('punkt')
stemmer = PorterStemmer()
masters = pd.read_json(r"C:\Users\Elias Antoun\Downloads\dataset.json")

[nltk_data] Downloading package stopwords to C:\Users\Elias
[nltk_data]     Antoun\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Elias
[nltk_data]     Antoun\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Preprocessing 2.0

In [11]:
# 2.0.0 Tokenize, remove stopwords and punctuation, and stem

stop_words = set(stopwords.words('english'))

def tok_punct_and_stop(text):
    desc_tok = word_tokenize(text)
    desc_tok_nostop_nopunct = [x for x in desc_tok if x.lower() not in stop_words and x not in string.punctuation]
    stemmed_desc = [stemmer.stem(x) for x in desc_tok_nostop_nopunct]
    return stemmed_desc

masters["cleaned_description"] = masters["description"].apply(tok_punct_and_stop)


In [12]:
masters

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,country,administration,url,cleaned_description
0,3D Design for Virtual Environments - MSc,Glasgow Caledonian University,School of Engineering and Built Environment,Full time,\n\n3D visualisation and animation play a role...,September,Please see the university website for further ...,MSc,1 year full-time,Glasgow,United Kingdom,On Campus,,"[3d, visualis, anim, play, role, mani, area, p..."
1,Accounting and Finance - MSc,University of Leeds,Leeds University Business School,Full time,\n\nBusinesses and governments rely on sound f...,September,"UK: £18,000 (Total) International: £34,750 (To...",MSc,1 year full time,Leeds,United Kingdom,On Campus,,"[busi, govern, reli, sound, financi, knowledg,..."
2,Accounting and Finance (MSc),University of Bath,School of Management,Full time,\n\nDevelop in-depth knowledge of accounting a...,September,\n</div,MSc,1 year full-time,Bath,United Kingdom,On Campus,,"[develop, in-depth, knowledg, account, financ,..."
3,"Accounting, Accountability & Financial Manag...",King’s College London,King’s Business School,Full time,"\n\nOur Accounting, Accountability & Financial...",September,Please see the university website for further ...,MSc,1 year FT,London,United Kingdom,On Campus,,"[account, account, financi, manag, msc, cours,..."
4,"Accounting, Financial Management and Digital...",University of Reading,Henley Business School,Full time,\n\nEmbark on a professional accounting career...,September,\n</div,MSc,1 year full time,Reading,United Kingdom,On Campus,,"[embark, profession, account, career, academ, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1825,Cancer Biology (MSc),Sheffield Hallam University,Postgraduate Courses,Full time,\n\nWork in state-of-the-art facilities and ga...,September,Please see the university website for further ...,MSc,"1 year full-time, 2 years part-time",Sheffield,United Kingdom,On Campus,,"[work, state-of-the-art, facil, gain, in-depth..."
1826,Cancer Biology and Immunology (MSc),University of Southampton,Faculty of Medicine,Full time,\n\nEmbark on a career in cancer research with...,See Course,Please see the university website for further ...,MSc,1 year full-time; 2 years part-time,Southampton,United Kingdom,On Campus,,"[embark, career, cancer, research, focu, lates..."
1827,Cancer Biology and Precision Oncology (Onlin...,University of Edinburgh,Edinburgh Medical School,Part time,\n\nProgramme descriptionThis new programme wi...,September,"For 2022/23 the fees are £6,035.",MSc,1-3 years part-time,Edinburgh,United Kingdom,Online,,"[programm, descriptionthi, new, programm, equi..."
1828,Cancer Biology and Therapeutics - MSc,Cardiff University,Cardiff School of Pharmacy and Pharmaceutical ...,Full time,\n\nWhy study this courseGraduates from this c...,September,Please see the university website for further ...,MSc,1 year full time,Cardiff,United Kingdom,On Campus,,"[studi, coursegradu, cours, breadth, depth, ca..."


In [13]:
# 2.0.1 fee preprocessing

# Step 1: Currency management setup


# Dictionary to standardize possible ways of writing a currency

curr_codes = {"EUR": ["euros", "€", "eur", "euro"],
              "USD": ["usd", "dollars", "$", "dollar"],
              "GBP": ["gbp", "pounds", "pound", "£"]}


# Step 2: amount and currency extraction
def extract_amount_and_curr(text):
    currencies = ["usd", "$", "dollars", "dollar", "euro", "euros", "€", "gbp", "eur", "£", "pounds"]
    amount = []
    i = 0
    text_tok = tok_punct_and_stop(text)
    tex_l = []
    for entry in text_tok:
        # Find the index of the first digit
        digit_index = next((i for i, c in enumerate(entry) if c.isdigit()), None)

        if digit_index is not None:
            # Split the entry into two parts
            prefix = entry[:digit_index]
            suffix = entry[digit_index:]

            # Add the parts to the result list
            tex_l.append(prefix)
            tex_l.append(suffix)
        else:
            # If no digit is found, keep the original entry
            tex_l.append(entry)

    # parse through obtained list to find numbers and currency symbols
    while i < len(tex_l):
        if tex_l[i].lower() in currencies:
            if i - 1 >= 0 and tex_l[i - 1].replace(',', '').isnumeric():
                amount.append(tex_l[i - 1].replace(',', ''))
            elif i + 1 < len(tex_l) and tex_l[i + 1].replace(',', '').isnumeric():
                amount.append(tex_l[i + 1].replace(',', ''))
            amount.append(tex_l[i])
        i += 1
    return amount


masters["fees"] = masters["fees"].apply(extract_amount_and_curr)


# Step 3: define function to return the max amount, and its currency

def max_standard(amount):
    curr = []
    price = [float(x) for x in amount if x.isnumeric()]
    for y in amount:
        for key, values in curr_codes.items():
            if y.lower() in values:
                curr.append(key)

    return sorted(price, reverse=True), curr


masters["fees"] = masters["fees"].apply(max_standard)

# Step 4: Conversion


def convert_to_euros(amount, source_currency):
    c = CurrencyRates()
    try:
        exchange_rate = c.get_rate(source_currency, 'EUR')
        return amount * exchange_rate
    except Exception as e:
        print(f"Error: {e}")
        return None

def max_amounts_in_euros(fees_column):
    max_amounts = []

    for entry in fees_column:
        amounts, currencies = entry
        max_amount_euros = float('-inf')

        for amount, currency in zip(amounts, currencies):
            amount_euros = convert_to_euros(amount, currency)
            if amount_euros is not None:
                max_amount_euros = max(max_amount_euros, amount_euros)

        max_amounts.append(max_amount_euros)

    return max_amounts

masters['fees'] = max_amounts_in_euros(masters['fees'])
masters['fees'] = masters['fees'].apply(lambda x: f"{x:.2f}" if pd.notna(x) else np.nan)
print(masters["fees"])

0           -inf
1       39871.49
2           -inf
3           -inf
4           -inf
          ...   
1825        -inf
1826        -inf
1827     6924.44
1828        -inf
1829    17325.45
Name: fees, Length: 1830, dtype: object


### Build a vocabulary

In [14]:
vocabulary = Counter(reduce(lambda x, y: x + y, masters.cleaned_description.values)).keys()
index = {}
unique_id = 1
for word in list(vocabulary):
  index[word] = unique_id
  unique_id+=1
index

{'3d': 1,
 'visualis': 2,
 'anim': 3,
 'play': 4,
 'role': 5,
 'mani': 6,
 'area': 7,
 'popular': 8,
 'media': 9,
 'keep': 10,
 'grow': 11,
 'digit': 12,
 'provid': 13,
 'eye-catch': 14,
 'special': 15,
 'effect': 16,
 '21st': 17,
 'centuri': 18,
 "'s": 19,
 'favourit': 20,
 'film': 21,
 'televis': 22,
 'show': 23,
 'design': 24,
 'also': 25,
 'essenti': 26,
 'everyday': 27,
 'work': 28,
 'everyth': 29,
 'comput': 30,
 'game': 31,
 'develop': 32,
 'onlin': 33,
 'virtual': 34,
 'world': 35,
 'industri': 36,
 'market': 37,
 'product': 38,
 'architecture.gcu': 39,
 'programm': 40,
 'environ': 41,
 'help': 42,
 'skill': 43,
 'thrive': 44,
 'success': 45,
 'career': 46,
 'visual': 47,
 'practic': 48,
 'career-focus': 49,
 'orient': 50,
 'toward': 51,
 'current': 52,
 'need': 53,
 'technolog': 54,
 'prior': 55,
 'knowledg': 56,
 'requir': 57,
 'busi': 58,
 'govern': 59,
 'reli': 60,
 'sound': 61,
 'financi': 62,
 'underpin': 63,
 'strategi': 64,
 'success.thi': 65,
 'cours': 66,
 'kick-start

In [15]:
terms = pd.DataFrame(data=list(vocabulary), columns=['term'])
terms['documents'] = terms.term.apply(lambda item: list(masters.loc[masters.cleaned_description.apply(lambda row: item in row)].index))

In [16]:
def find_word_in_documents(word):
    filtered_rows = terms[terms['term'] == word]['documents'].values[0]
    result_df = masters.loc[filtered_rows, ['courseName', 'universityName', 'description', 'url']]
    return result_df

word_to_search = "account"
f_n_words = find_word_in_documents(word_to_search)
f_n_words

,courseName,universityName,description,url
1,Accounting and Finance - MSc,University of Leeds,\n\nBusinesses and governments rely on sound f...,
2,Accounting and Finance (MSc),University of Bath,\n\nDevelop in-depth knowledge of accounting a...,
3,"Accounting, Accountability & Financial Manag...",King’s College London,"\n\nOur Accounting, Accountability & Financial...",
4,"Accounting, Financial Management and Digital...",University of Reading,\n\nEmbark on a professional accounting career...,
55,Cardiovascular Sciences - MSc(MedSci),University of Glasgow,\n\nCardiovascular disease is projected to rem...,
...,...,...,...,...
1699,Building Services Engineering Management MSc,Brunel University London,\n\nAsk BrunelBrunel's Building Services Engin...,
1714,Business Administration MSc,University of Groningen,\n\nStudy business administration in Groningen...,
1720,Business Analysis & Finance - MSc,University of Leicester,\n\nThe Business Analysis and Finance MSc from...,
1727,Business Analytics - MSc,Xi’an Jiaotong-Liverpool University,\n\nThe rapid growth of available data is tran...,


**2.2**

In [17]:
tfidf = TfidfVectorizer()
results = tfidf.fit_transform(masters.description)
termss = tfidf.get_feature_names_out()

In [18]:
tfidf_matrix = results
feature_names = termss

In [19]:
def create_inverted_indexes(tfidf_matrix, features_names):
    inverted_indexes = defaultdict(list)
    for term_id, term in enumerate(features_names):
        term_indices = tfidf_matrix[:, term_id].nonzero()[0]
        for document_index in term_indices:
            tfidf_value = tfidf_matrix[document_index, term_id]
            inverted_indexes[term_id].append((document_index, tfidf_value))
    return inverted_indexes

inverted_indexes = create_inverted_indexes(tfidf_matrix, feature_names)

In [20]:
term_to_check = "account"
term_id_to_check = np.where(feature_names == term_to_check)[0][0]
print(f"{term_to_check}: {inverted_indexes[term_id_to_check]}")

account: [(727, 0.15978417968994663), (757, 0.19568348535764485), (1258, 0.2669691823999831)]


In [21]:
word_to_search = 'advanced knowledge'
word_tfidf = tfidf.transform([word_to_search])
cosine_s = cosine_similarity(word_tfidf, results).flatten()
point_2_2df = pd.DataFrame({
    'courseName': masters['courseName'],
    'universityName': masters['universityName'],
    'description': masters['description'],
    'url': masters['url'],
    'Similarity': cosine_s})
point_2_2df = point_2_2df.sort_values(by='Similarity', ascending = False)
point_2_2df.head(10)

,courseName,universityName,description,url,Similarity
977,Advanced Software Engineering MSc,King’s College London,\n\nOur Advanced Software Engineering MSc is a...,,0.361762
769,Advanced Clinical Practice MSc,University of Greenwich,\n\nLearn essential strategies and prepare for...,,0.330062
723,Advanced Clinical Practice - MSc,Canterbury Christ Church University,\n\nGain the knowledge and skills needed to be...,,0.310597
824,Advanced Computing MSc,King’s College London,\n\nOur Advanced Computing MSc provides knowle...,,0.308847
776,Advanced Clinical Practice MSc/PGDip,Middlesex University,\n\nOur Advanced Clinical Practice (ACP) degre...,,0.298787
856,Advanced Healthcare Practice MSc,Liverpool John Moores University,\n\nDeveloped in response to the current Advan...,,0.287418
898,Advanced Mechanical Engineering - MSc (Eng),University of Leeds,\n\nThis course offers a broad range of advanc...,,0.267400
780,Advanced Clinical Practitioner - MSc,University of Sunderland,\n\nThe MSc Advanced Clinical Practitioner is ...,,0.264792
854,Advanced Healthcare Practice - MSc,Cardiff University,\n\nWhy study this courseOur MSc Advanced Heal...,,0.257279
739,Advanced Clinical Practice (AHP) - MSc/PGDip...,Bangor University,\n\nThe programme has been developed to enhanc...,,0.255845


**PART 3: ALternative Scoring function**

In this part we will be implementing an alternative, and more rigorous scoring function compared to the one presented in 2.2.

This function will be scoring the reuslts accordidng to the following criteria:

1- Description matching as before
2- Exact matches in course titles
3- Possible matches in geographical location
4- Possible tuition fee matching, preferring universities with tuition costs lower than any that may be specified in the query.
5- Possible matches in modality and administration
6- Preference to higher ranked universities (Qs 2024 ranking)

*Score 1: Description matching*

In [40]:
def description_score(query_tok):
    description_terms = masters["cleaned_description"]
    match_count = description_terms.apply(lambda x: sum(token in x for token in query_tok))
    return match_count

*Score 2: Exact Matches in Course Titles*

In [25]:
def title_exact_match_score(query_l):
    title_matches = masters[masters['courseName'].str.lower().isin(query_l)]['courseName'].tolist()
    return 1 if  title_matches else 0

*Score 3: Geographical location*

In [26]:
def location_score(query_l):
    matching_locations = masters[
    (masters['city'].str.lower().isin(query_l)) &
    (masters['country'].str.lower().isin(query_l))
]['city'].tolist()
    return 1 if matching_locations else 0

*Score 4: Tuition fees matching*

Only works if masters["fees"] is reported in the dataset

In [58]:
def fee_score(user_budget):
    return 1 if int(user_budget) >= masters["fees"] else 0

Score 5: Administration score

In [28]:
def administration_score(query_l):
    admin_match = masters[masters["administration"].str.lower().isin(qeury_l)]["administration"].tolist()
    return 1 if admin_match else 0

Score 6: Duration score

In [29]:
def clean_duration(duration):
    # Extract numeric values and keywords (full time, part time) for comparison
    duration_keywords = re.findall(r'\b(?:year|full time|part time)\b', duration.lower())
    duration_values = re.findall(r'\d+', duration)
    # Combine numeric values and keywords
    cleaned_duration = ' '.join(duration_values + duration_keywords)

    return cleaned_duration


def duration_score(user_duration):
    # Clean and standardize the duration entries for comparison
    cleaned_user_duration = clean_duration(user_duration)
    cleaned_degree_duration = clean_duration(masters["duration"])
    

    # Check for an exact match
    if cleaned_user_duration == cleaned_degree_duration:
        return 1
    # Check for a partial match (e.g., "1 year" in "1 year full time")
    elif cleaned_user_duration in cleaned_degree_duration:
        return 0.8
    # Check for any overlap between the user and degree durations
    elif has_overlap(cleaned_user_duration, cleaned_degree_duration):
        return 0.6
    else:
        return 0



    

def has_overlap(duration1, duration2):
    # Check if there is any overlap between the cleaned durations
    return any(keyword in duration2 for keyword in duration1.split())

Overall Score

Step 2: Scoring functions and weights

In [59]:
def overall_score(query, masters):
 
    query_tok = tok_punct_and_stop(query) #for budget and description matching
    query_l = word_tokenize(query) # for exact matches search
    for element in query_tok:
        if element.isnumeric():
            budget = element

    desc_score = description_score(query_tok)
    title_exact_match = title_exact_match_score(query_l)
    location_score_value = location_score(query_l)
    fee_score_value = fee_score(budget)
    admin_score_value = administration_score(query_l)
    #duration_score_value = duration_score(user_duration)

    total_score = (
        (desc_score * 0.4) +
        (title_exact_match * 0.4) +
        (location_score_value * 0.2) +
        (fee_score_value * 0.2) +
        (admin_score_value * 0.2)
    )

    return total_score

**Score**

In [60]:
user_query = "data science full time in London under 30000 online for 1 year"

score = overall_score(user_query, masters)


point_2_2df = pd.DataFrame({
    'courseName': masters['courseName'],
    'universityName': masters['universityName'],
    'description': masters['description'],
    'url': masters['url'],
    'score': score})

KeyError: 'fees'